In [1]:
import os
import os.path as osp
import sys

import numpy as np
import torch

from NeuralNetwork.network import NNClassifier
from NeuralNetwork.dataset import MyDataset


EXPERIMENT = 'baseline'
CHPT = 144
SAVE_NAME = 'nnclassifier'

ckpt_file = f'output/checkpoints/{EXPERIMENT}/epoch={CHPT}.ckpt'
weights_file = f'models/{SAVE_NAME}.pth'

ckpt = torch.load(ckpt_file, map_location='cpu')
state_dict = {k[9:] if k.startswith('backbone.') else k: v for k, v in ckpt['state_dict'].items()}
torch.save({'model': state_dict}, weights_file)

X, features = MyDataset.load_data('data/test.csv', test=True)
X = torch.from_numpy(X).float()

model = NNClassifier()
ckpt = torch.load(weights_file, map_location='cpu')
model.load_state_dict(ckpt['model'])

<All keys matched successfully>

In [2]:
def predict(logits):
    return torch.argmax(logits, dim=1).numpy()

preds = predict(model(X))

In [4]:
with open('output/answer.csv', 'w') as f:
    f.write('Id,Category\n')
    for i, pred in enumerate(preds):
        string = f"{i},{int(pred)}\n"
        f.write(string)